In [70]:
import pandas as pd
import numpy as np
demo = pd.read_csv('demo.csv')

In [210]:
test_df = pd.read_csv("test.csv")
test_df

,PANEL_FLAG,SHEET_ID_CHIP_ID,SHEET_ID,LAYOUT_OX,SHEET_X_CHIP_CNT,SHEET_Y_CHIP_CNT,SUBSHEET_X_CHIP_CNT,SUBSHEET_Y_CHIP_CNT,START_CHIP_QTY,Unnamed: 9,Unnamed: 10
0,S,EE2N23T4A0,EE2N23Z4A,XXXXXXXXXXXXXXXXXXOOOOOOOOOOOOOOOOOO,9,4,3,2,36,NaN,NaN
1,M,EE2N23T4A0CB,EE2N23Z4A,XXXOOO,9,4,3,2,6,NaN,NaN
2,S,ED01N92810P03,ED01N92810P03,XXXXXXXXXOXXXOXXXOOOOXOXOOXOXXXXOOOOOOOXXXXOXXOX,8,6,4,2,48,ED01N92810P,3.0
3,M,ED01N92810T0331,ED01N92810P03,XXXXXOXX,8,6,4,2,8,ED01N92810T03,31.0


In [212]:
def pre_processing(df):
    M_CHIP = df[['PANEL_FLAG', 'SHEET_ID_CHIP_ID', 'SHEET_ID', 'LAYOUT_OX', 'SHEET_X_CHIP_CNT', 'SHEET_Y_CHIP_CNT', 'SUBSHEET_X_CHIP_CNT', 'SUBSHEET_Y_CHIP_CNT', 'START_CHIP_QTY']]
    M_CHIP['SHEET_ID_CHIP'] = [x[:-2] for x in M_CHIP['SHEET_ID_CHIP_ID'].tolist()]
    M_CHIP['INDEX'] = [x[-2:] for x in M_CHIP['SHEET_ID_CHIP_ID'].tolist()]  
    return M_CHIP

M_CHIP = pre_processing(test_df)
# M_CHIP.to_csv('M_CHIP.csv')
M_CHIP

C:\Users\BarryShih\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PANEL_FLAG,SHEET_ID_CHIP_ID,SHEET_ID,LAYOUT_OX,SHEET_X_CHIP_CNT,SHEET_Y_CHIP_CNT,SUBSHEET_X_CHIP_CNT,SUBSHEET_Y_CHIP_CNT,START_CHIP_QTY,SHEET_ID_CHIP,INDEX
0,S,EE2N23T4A0,EE2N23Z4A,XXXXXXXXXXXXXXXXXXOOOOOOOOOOOOOOOOOO,9,4,3,2,36,EE2N23T4,A0
1,M,EE2N23T4A0CB,EE2N23Z4A,XXXOOO,9,4,3,2,6,EE2N23T4A0,CB
2,S,ED01N92810P03,ED01N92810P03,XXXXXXXXXOXXXOXXXOOOOXOXOOXOXXXXOOOOOOOXXXXOXXOX,8,6,4,2,48,ED01N92810P,03
3,M,ED01N92810T0331,ED01N92810P03,XXXXXOXX,8,6,4,2,8,ED01N92810T03,31


In [221]:
def slice_m_chip(df):
    alpha = list('abcdefghijklmnopqrstuvwxyz'.upper())
    df = df.reset_index(drop=True)
    df_result = pd.DataFrame(data = [], columns=df.columns.tolist())
    
    
    for i in range(len(df.index)):
        flag_6k = False
        cur_df = df.iloc[i]
        if cur_df['PANEL_FLAG'] == 'S':
            if len(cur_df['SHEET_ID']) < 10:
                flag_6k = True
            x_length = int(cur_df['SHEET_X_CHIP_CNT'])
            y_length = int(cur_df['SHEET_Y_CHIP_CNT'])
            ins_orign = list(cur_df['LAYOUT_OX'])
            SHEET_ID_CHIP_ID = cur_df['SHEET_ID_CHIP_ID']
            
            test_dict = {}
            for y in range(y_length):
                for x in range(x_length):
                    test_dict[x,y] = ins_orign[0]
                    ins_orign.pop(0)
            
            sub_id = []
            sub_ins = []
            
            for key in test_dict.keys():
                tmp = ''
                for i in list(key):
                    tmp = tmp + str(i)
                sub_id.append(SHEET_ID_CHIP_ID + tmp)
                sub_ins.append(test_dict[key])
                
            if flag_6k == True:
                tmp = []
                for i in sub_id:
                    tmp.append(i[:-2] + alpha[int(i[-2])] + alpha[int(i[-1])])
                sub_id = tmp
                
        
            df_sub = pd.DataFrame(data = [], columns=df.columns.tolist())
            df_sub['SHEET_ID_CHIP_ID'] = sub_id
            df_sub['LAYOUT_OX'] = sub_ins
            df_sub['PANEL_FLAG'] = cur_df['PANEL_FLAG']
            df_sub['SHEET_ID'] = cur_df['SHEET_ID']
            df_sub['SHEET_X_CHIP_CNT'] = cur_df['SHEET_X_CHIP_CNT']
            df_sub['SHEET_Y_CHIP_CNT'] = cur_df['SHEET_Y_CHIP_CNT']
            df_sub['SUBSHEET_X_CHIP_CNT'] = cur_df['SUBSHEET_X_CHIP_CNT']
            df_sub['SUBSHEET_Y_CHIP_CNT'] = cur_df['SUBSHEET_Y_CHIP_CNT']
            df_sub['START_CHIP_QTY'] = 1            
            df_result = pd.concat([df_result, df_sub], ignore_index=True)
        
        
        elif cur_df['PANEL_FLAG'] == 'M':
            if len(cur_df['SHEET_ID']) < 10:
                flag_6k = True
                tmp = cur_df['SHEET_ID_CHIP_ID'][:-2] + str(alpha.index(cur_df['SHEET_ID_CHIP_ID'][-2])) + str(alpha.index(cur_df['SHEET_ID_CHIP_ID'][-1])) 
                cur_df['SHEET_ID_CHIP_ID'] = tmp

                
            SHEET_ID_CHIP_ID = cur_df['SHEET_ID_CHIP_ID'][:-2]
            ins_orign = list(cur_df['LAYOUT_OX'])
            end_index_list = list(cur_df['SHEET_ID_CHIP_ID'][-2:])
            start_index = [int(end_index_list[0])-cur_df['SUBSHEET_X_CHIP_CNT']+1, int(end_index_list[1])-cur_df['SUBSHEET_Y_CHIP_CNT']+1]
            x_length = int(cur_df['SUBSHEET_X_CHIP_CNT'])
            y_length = int(cur_df['SUBSHEET_Y_CHIP_CNT'])

            
            test_dict = {}
            for y in range(y_length):
                for x in range(x_length):
                    cur_x = x + start_index[0]
                    cur_y = y + start_index[1]
                    test_dict[cur_x,cur_y] = ins_orign[0]
                    ins_orign.pop(0)
            
            sub_id = []
            sub_ins = []
            
            for key in test_dict.keys():
                tmp = ''
                for i in list(key):
                    tmp = tmp + str(i)
                sub_id.append(SHEET_ID_CHIP_ID + tmp)
                sub_ins.append(test_dict[key])
                

            if flag_6k == True:
                tmp = []
                for i in sub_id:

                    tmp.append(i[:-2] + alpha[int(i[-2])] + alpha[int(i[-1])])
                sub_id = tmp        
        
            df_sub = pd.DataFrame(data = [], columns=df.columns.tolist())
            df_sub['SHEET_ID_CHIP_ID'] = sub_id
            df_sub['LAYOUT_OX'] = sub_ins
            df_sub['PANEL_FLAG'] = cur_df['PANEL_FLAG']
            df_sub['SHEET_ID'] = cur_df['SHEET_ID']
            df_sub['SHEET_X_CHIP_CNT'] = cur_df['SHEET_X_CHIP_CNT']
            df_sub['SHEET_Y_CHIP_CNT'] = cur_df['SHEET_Y_CHIP_CNT']
            df_sub['SUBSHEET_X_CHIP_CNT'] = cur_df['SUBSHEET_X_CHIP_CNT']
            df_sub['SUBSHEET_Y_CHIP_CNT'] = cur_df['SUBSHEET_Y_CHIP_CNT']
            df_sub['START_CHIP_QTY'] = 1
            df_result = pd.concat([df_result, df_sub], ignore_index=True)
    
    return df_result

# ck_01 = M_CHIP.loc[[1],['PANEL_FLAG', 'SHEET_ID_CHIP_ID', 'SHEET_ID', 'LAYOUT_OX', 'SHEET_X_CHIP_CNT', 'SHEET_Y_CHIP_CNT', 'SUBSHEET_X_CHIP_CNT', 'SUBSHEET_Y_CHIP_CNT', 'START_CHIP_QTY']]
ck_01 = M_CHIP[['PANEL_FLAG', 'SHEET_ID_CHIP_ID', 'SHEET_ID', 'LAYOUT_OX', 'SHEET_X_CHIP_CNT', 'SHEET_Y_CHIP_CNT', 'SUBSHEET_X_CHIP_CNT', 'SUBSHEET_Y_CHIP_CNT', 'START_CHIP_QTY']]
slice_m_chip(ck_01)

C:\Users\BarryShih\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,PANEL_FLAG,SHEET_ID_CHIP_ID,SHEET_ID,LAYOUT_OX,SHEET_X_CHIP_CNT,SHEET_Y_CHIP_CNT,SUBSHEET_X_CHIP_CNT,SUBSHEET_Y_CHIP_CNT,START_CHIP_QTY
0,S,EE2N23T4A0AA,EE2N23Z4A,X,9,4,3,2,1
1,S,EE2N23T4A0BA,EE2N23Z4A,X,9,4,3,2,1
2,S,EE2N23T4A0CA,EE2N23Z4A,X,9,4,3,2,1
3,S,EE2N23T4A0DA,EE2N23Z4A,X,9,4,3,2,1
4,S,EE2N23T4A0EA,EE2N23Z4A,X,9,4,3,2,1
...,...,...,...,...,...,...,...,...,...
93,M,ED01N92810T0330,ED01N92810P03,X,8,6,4,2,1
94,M,ED01N92810T0301,ED01N92810P03,X,8,6,4,2,1
95,M,ED01N92810T0311,ED01N92810P03,O,8,6,4,2,1
96,M,ED01N92810T0321,ED01N92810P03,X,8,6,4,2,1


In [ ]:
# def slice_m_chip(df):
#     SHEET_ID_CHIP = list(set(df['SHEET_ID_CHIP']))
#     column_name = df.columns.tolist()[:-2]
#     df_sub_all = pd.DataFrame(data = [], columns=column_name)
#     print('SHEET_ID_CHIP: ', SHEET_ID_CHIP)
    
#     for cur_sheet_id_chip in SHEET_ID_CHIP:
#         cur_df = df[df['SHEET_ID_CHIP'].astype(str).str.contains(cur_sheet_id_chip)]
#         index_orgin = [list(x) for x in df[df['SHEET_ID_CHIP'].astype(str).str.contains(cur_sheet_id_chip)]['INDEX'].tolist()]
#         ins_orgin = [x for x in df[df['SHEET_ID_CHIP'].astype(str).str.contains(cur_sheet_id_chip)]['LAYOUT_OX'].tolist()]
#         axis_x_max = max([x[0] for x in index_orgin])
#         axis_y_max = max([x[1] for x in index_orgin])
        
#         print('cur_sheet_id_chip: ', cur_sheet_id_chip)
#         print("index_orgin: ", index_orgin)
#         print("ins_orgin: ", ins_orgin)
#         print("axis_x_max, axis_y_max: ", axis_x_max, ",",axis_y_max)
        
#         x_axis_type = list(set([int(x[0]) for x in index_orgin]))
#         print(x_axis_type)
#         x_length = (max(x_axis_type)-min(x_axis_type))/ (len(x_axis_type)-1)

#         y_axis_type = list(set([int(x[1]) for x in index_orgin]))
#         y_length = (max(y_axis_type)-min(y_axis_type))/ (len(y_axis_type)-1)
#         print("x_length: ", x_length, "y_length: ", y_length)

#         total_size = (int(axis_x_max)+1) * (int(axis_y_max)+1)
#         print('total_size: ', total_size)

#         chip_arr = np.full([int(axis_y_max)+1,int(axis_x_max)+1], "1")
#         print("chip_arr: ", chip_arr)
        
#         # reverse ox
#         # ins_orgin = [list(x[::-1]) for x in ins_orgin]

#         test_dict = {}

#         for i in range(len(index_orgin)):

#             cnt = 0
#             for y in range(int(y_length)):

#                 cur_y = int(index_orgin[i][1]) - y

#                 for x in range(int(x_length)):

#                     cur_x = int(index_orgin[i][0]) - x
#                     chip_arr[int(axis_y_max)-cur_y][cur_x] = ins_orgin[i][cnt]
#                     test_dict[cur_x,cur_y] = ins_orgin[i][cnt]
#                     cnt+=1
#         print(chip_arr)
    
#         sub_id = []
#         sub_ins = []
#         for key in test_dict.keys():
#             tmp = ''
#             for i in list(key):
#                 tmp = tmp + str(i)
#             sub_id.append(cur_sheet_id_chip + tmp)
#             sub_ins.append(test_dict[key])
#         print('sub_id: ', sub_id)
#         print('sub_id_num: ', len(sub_id))
#         print('sub_ins: ', sub_ins)
#         print('sub_ins_num: ', len(sub_ins))
        
#         df_sub = pd.DataFrame(data = [], columns=column_name)
#         df_sub['SHEET_ID_CHIP_ID'] = sub_id
#         df_sub['LAYOUT_OX'] = sub_ins
#         df_sub['PANEL_FLAG'] = cur_df['PANEL_FLAG'].tolist()[0]
#         df_sub['SHEET_ID'] = cur_df['SHEET_ID'].tolist()[0]
#         df_sub['SHEET_X_CHIP_CNT'] = cur_df['SHEET_X_CHIP_CNT'].tolist()[0]
#         df_sub['SHEET_Y_CHIP_CNT'] = cur_df['SHEET_Y_CHIP_CNT'].tolist()[0]
#         df_sub['SUBSHEET_X_CHIP_CNT'] = cur_df['SUBSHEET_X_CHIP_CNT'].tolist()[0]
#         df_sub['SUBSHEET_Y_CHIP_CNT'] = cur_df['SUBSHEET_Y_CHIP_CNT'].tolist()[0]
#         df_sub['START_CHIP_QTY'] = 1
        
#         df_sub_all = pd.concat([df_sub_all, df_sub], ignore_index=True)
#         break
#     return df_sub_all
        
        
        
        
# df_sub_all = slice_m_chip(M_CHIP)
# df_sub_all
# df_sub_all.to_csv('df_sub_all.csv', index = False)